# global annotations file download wrangle~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- downloaded from: 
    - GRCh38_latest_genomic.gff.gz : 
        - https://www.ncbi.nlm.nih.gov/genome/guide/human/
        - "RefSeq Reference Genome Annotation" "GRCh38"
    - ncbi refseq -->chr# conversion 
        - https://hgdownload.soe.ucsc.edu/goldenPath/hg38/database/
        - description of this file: (at https://hgdownload.cse.ucsc.edu/goldenpath/hg38/bigZips/) 
            - Sequence names^^^^^^^^^^^^^^
            - For historical reasons, what UCSC calls "chr1", Ensembl calls "1" and NCBI calls "NC_000067.6". The sequences are identical though. To map between UCSC, Ensembl and NCBI names, use our table "chromAlias", available via our Table Browser or as file: 
                - https://hgdownload.cse.ucsc.edu/goldenPath/mm10/database/chromAlias.txt.gz
        - wrong link to file, so replaced "mm10" (mouse) to the human hg38
    - gencode.v39.annotation.gff3.gz : 
        - https://www.gencodegenes.org/human/
        - "Comprehensive gene annotation" "Gff3" 
- download date: 
    - check the wget output 
        - wget_output.txt = ncbi 
        - wget_output1.txt = gencode 
- genome build 
    - both GRCh38 --> converto to hg18. 
- file format & head: 


In [1]:
# !mkdir ../../../../data/global/track_data/annotation/

In [2]:
%%bash
cd ../../../../
wget https://ftp.ncbi.nlm.nih.gov/refseq/H_sapiens/annotation/GRCh38_latest/refseq_identifiers/GRCh38_latest_genomic.gff.gz -P data/global/track_data/annotation/ -o data/global/track_data/annotation/wget_ouput.txt
    

In [21]:
%%bash 
cd ../../../../
zcat data/global/track_data/annotation/GRCh38_latest_genomic.gff.gz | grep -v "#" | awk '{print $3}' | sort | uniq -c > data/global/track_data/annotation/GRCh38_latest_genomic_summary.txt

In [17]:
%%bash 
cd ../../../../
wget https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_39/gencode.v39.annotation.gff3.gz -P data/global/track_data/annotation/ -o data/global/track_data/annotation/wget_ouput1.txt

In [22]:
%%bash 
cd ../../../../
zcat data/global/track_data/annotation/gencode.v39.annotation.gff3.gz | grep -v "#" | awk '{print $3}' | sort | uniq -c > data/global/track_data/annotation/gencode.v39.annotation_summary.txt

## **converting the ncbi refseq into chr#**

In [33]:
!wget https://hgdownload.soe.ucsc.edu/goldenPath/hg38/database/chromAlias.txt.gz  -P ../../../../data/global/track_data/annotation/

--2022-03-28 16:33:59--  https://hgdownload.soe.ucsc.edu/goldenPath/hg38/database/chromAlias.txt.gz
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12065 (12K) [application/x-gzip]
Saving to: ‘../../../../data/global/track_data/annotation/chromAlias.txt.gz’

100%[======================================>] 12,065      --.-K/s   in 0s      

2022-03-28 16:34:00 (84.5 MB/s) - ‘../../../../data/global/track_data/annotation/chromAlias.txt.gz’ saved [12065/12065]



In [1]:
import gzip
with gzip.open("../../../../data/global/track_data/annotation/GRCh38_latest_genomic.gff.gz", "rt") as handle:
        ncbi_annotation_lines = handle.readlines()
with gzip.open("../../../../data/global/track_data/annotation/chromAlias.txt.gz", "rt") as handle:
        conversion_lines = handle.readlines()

In [2]:
conversion_dict = {}
for line in conversion_lines: 
    if "refseq" in line:
        conversion_dict[line.split()[0]] = line.split()[1]
del(conversion_lines)

In [3]:
assert len(set(list(conversion_dict.values()))) == len(conversion_dict.values())# no duplicates
assert len(set(list(conversion_dict.keys()))) == len(conversion_dict.keys())#no duplicates 

In [4]:
from datetime import datetime
timestamp = datetime.now().strftime("%Y/%m/%d").replace("/", "_").replace(":", "_")
with open('../../../../data/global/track_data/annotation/GRCh38_latest_genomic_converted.gff'.format(t=timestamp), 'w') as f: 
    for i,line in enumerate(ncbi_annotation_lines): 
        if line[0] != "#" : # i already double checked that this doesnt leave out some annotation lines
            line_split = line.split("\t")
            refseq,source,feature,start,end,value_1,strand,value_2,ID = line_split
            chr_n = conversion_dict[refseq]
            string_to_write = chr_n+"\t"+source+"\t"+feature+"\t"+start+"\t"+end+"\t"+value_1+"\t"+strand+"\t"+value_2+"\t"+refseq+"\t"+ID
            string_to_write = string_to_write.replace(" ","_")
            f.write(string_to_write)

# **combine**

In [7]:
! cat ../../../../data/global/track_data/annotation/GRCh38_latest_genomic_converted.gff > ../../../../data/global/track_data/annotation/global_annotation.gff


In [11]:
%%bash 
cd ../../../../
zcat data/global/track_data/annotation/gencode.v39.annotation.gff3.gz | grep -v "#" >> data/global/track_data/annotation/global_annotation.gff

In [13]:
%%bash 
cd ../../../../
sort -k1,1 -k4,4n data/global/track_data/annotation/global_annotation.gff >  data/global/track_data/annotation/global_annotation_sorted.gff

In [14]:
%%bash 
cd ../../../../
bgzip data/global/track_data/annotation/global_annotation_sorted.gff

In [15]:
%%bash 
cd ../../../../
tabix -p gff data/global/track_data/annotation/global_annotation_sorted.gff.gz

In [19]:
%%bash 
cd ../../../../
tabix  data/global/track_data/annotation/global_annotation_sorted.gff.gz chr1:1000-10000

chr1	RefSeq	region	1	248956422	.	+	.	NC_000001.11	ID=NC_000001.11:1..248956422;Dbxref=taxon:9606;Name=1;chromosome=1;gbkey=Src;genome=chromosome;mol_type=genomic_DNA


In [1]:
! zcat ../../../../data/global/track_data/annotation/global_annotation_sorted.gff.gz | head 

chr1	RefSeq	region	1	248956422	.	+	.	NC_000001.11	ID=NC_000001.11:1..248956422;Dbxref=taxon:9606;Name=1;chromosome=1;gbkey=Src;genome=chromosome;mol_type=genomic_DNA
chr1	HAVANA	exon	11869	12227	.	+	.	ID=exon:ENST00000456328.2:1;Parent=ENST00000456328.2;gene_id=ENSG00000223972.5;transcript_id=ENST00000456328.2;gene_type=transcribed_unprocessed_pseudogene;gene_name=DDX11L1;transcript_type=processed_transcript;transcript_name=DDX11L1-202;exon_number=1;exon_id=ENSE00002234944.1;level=2;transcript_support_level=1;hgnc_id=HGNC:37102;tag=basic;havana_gene=OTTHUMG00000000961.2;havana_transcript=OTTHUMT00000362751.1
chr1	HAVANA	gene	11869	14409	.	+	.	ID=ENSG00000223972.5;gene_id=ENSG00000223972.5;gene_type=transcribed_unprocessed_pseudogene;gene_name=DDX11L1;level=2;hgnc_id=HGNC:37102;havana_gene=OTTHUMG00000000961.2
chr1	HAVANA	transcript	11869	14409	.	+	.	ID=ENST00000456328.2;Parent=ENSG00000223972.5;gene_id=ENSG00000223972.5;transcript_id=ENST00000456328.2;gene_type=transcribed_unprocessed_

In [ ]:
[RNA|codon|transcript|gene|CDS|exon|cDNA|CDS]

In [4]:
%%bash 
cd ../../../../data/global/track_data/annotation/
zcat global_annotation_sorted.gff.gz | sed 's/\t/,/g' > global_annotation_sorted.csv

In [16]:
%%bash 
cd ../../../../data/global/track_data/annotation/
zcat global_annotation_sorted.gff.gz | grep -E "chr[0-9]{1,2},.{6,20},telomerase_RNA|lnc_RNA|mRNA|ncRNA|miRNA|tRNA|vault_RNA|antisense_RNA|V_gene_segment|Y_RNA|RNase_MRP_RNA|RNase_P_RNA|rRNA|scRNA|snoRNA|snRNA|telomerase_RNA|start_codon|stop_codon|stop_codon_redefined_as_selenocysteine|transcript|primary_transcript|gene|D_gene_segment|J_gene_segment|C_gene_segment|CDS|exon|cDNA_match|CDS" > gene_annotation.csv 

In [17]:
%%bash 
cd ../../../../data/global/track_data/annotation/
 awk '{print $3}' gene_annotation.csv| sort | uniq -c 

     24 antisense_RNA
     19 biological_region
      1 CAAT_signal
     93 CAGE_cluster
  22914 cDNA_match
2340298 CDS
     24 centromere
     43 C_gene_segment
     61 D_gene_segment
     12 DNaseI_hypersensitive_site
   4546 enhancer
     55 enhancer_blocking_element
     12 epigenetically_modified_region
3560714 exon
 166248 five_prime_UTR
     14 GC_rich_promoter_region
 104385 gene
      2 imprinting_control_region
      6 insulator
    128 J_gene_segment
  28864 lnc_RNA
      7 locus_control_region
      1 microsatellite
   3184 miRNA
     16 mitotic_recombination_region
    188 mobile_genetic_element
 122549 mRNA
     31 ncRNA
     11 non_allelic_homologous_recombination_region
      1 nucleotide_motif
     30 origin_of_replication
   2119 primary_transcript
    410 promoter
    566 protein_binding_site
  18363 pseudogene
    598 recombination_feature
      2 repeat_instability_region
      3 replication_regulatory_region
      2 replication_start_site
     19 response_element


In [5]:
test1 = open("../../../../data/global/track_data/annotation/global_annotation_sorted.csv").readlines()[0:10]

In [6]:
import re 

In [7]:
test1[0]

'chr1,RefSeq,region,1,248956422,.,+,.,NC_000001.11,ID=NC_000001.11:1..248956422;Dbxref=taxon:9606;Name=1;chromosome=1;gbkey=Src;genome=chromosome;mol_type=genomic_DNA\n'

In [13]:
re.findall(r"chr[0-9]{1,2},.{6,20},region",test1[0])

['chr1,RefSeq,region']